## Imports and configuration


In [5]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import random
import math

# Optional geo imports (only needed if you want to plot real county shapes)
try:
    import geopandas as gpd
    GEO_AVAILABLE = True
except Exception:
    GEO_AVAILABLE = False

# Set random seed for reproducible layout/colors
random.seed(42)


## Load dataset and inspect


In [6]:
# Replace 'us_county_adjacency.csv' with the path to your CSV if it's in a different folder.
csv_path = './Dataset/us_county_adjacency.csv'
df = pd.read_csv(csv_path)

# Inspect first rows
print("Dataset shape:", df.shape)
display(df.head(10))

# Expect columns like: countryname, fipscounty, neighborname, fipsneighbor
print("\nColumns:", df.columns.tolist())


Dataset shape: (22200, 4)


,countyname,fipscounty,neighborname,fipsneighbor
0,"Autauga County, AL",1001,"Autauga County, AL",1001
1,"Autauga County, AL",1001,"Chilton County, AL",1021
2,"Autauga County, AL",1001,"Dallas County, AL",1047
3,"Autauga County, AL",1001,"Elmore County, AL",1051
4,"Autauga County, AL",1001,"Lowndes County, AL",1085
5,"Autauga County, AL",1001,"Montgomery County, AL",1101
6,"Baldwin County, AL",1003,"Baldwin County, AL",1003
7,"Baldwin County, AL",1003,"Clarke County, AL",1025
8,"Baldwin County, AL",1003,"Escambia County, AL",1053
9,"Baldwin County, AL",1003,"Mobile County, AL",1097



Columns: ['countyname', 'fipscounty', 'neighborname', 'fipsneighbor']


## Build adjacency graph from CSV


In [7]:

G = nx.Graph()

# Replace these with the actual names you found if they differ
col_a = "fipscounty"
col_a_name = "countyname"   # replace 'countryname' with 'countyname'
col_b = "fipsneighbor"
col_b_name = "neighborname" # replace if needed

# Add nodes with names
unique_nodes = set(df[col_a].unique()).union(set(df[col_b].unique()))
for n in unique_nodes:
    # find a human-friendly name if present
    name_row = df.loc[df[col_a] == n, col_a_name]
    if not name_row.empty:
        node_name = name_row.iloc[0]
    else:
        # try neighborname
        name_row2 = df.loc[df[col_b] == n, col_b_name]
        node_name = name_row2.iloc[0] if not name_row2.empty else str(n)
    G.add_node(n, display_name=str(node_name))

# Add edges (undirected)
for _, row in df.iterrows():
    a = row[col_a]
    b = row[col_b]
    if pd.isna(a) or pd.isna(b): 
        continue
    if a == b:
        continue
    G.add_edge(a, b)

print("Graph built. Nodes:", G.number_of_nodes(), "Edges:", G.number_of_edges())

# Show degree summary (helps identify disconnected components)
deg = sorted(G.degree(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 nodes by degree:", deg)


Graph built. Nodes: 3234 Edges: 9483
Top 10 nodes by degree: [(np.int64(49037), 14), (np.int64(32031), 13), (np.int64(51041), 11), (np.int64(31031), 11), (np.int64(51059), 10), (np.int64(35037), 10), (np.int64(53075), 10), (np.int64(8059), 10), (np.int64(25027), 10), (np.int64(12105), 10)]


## CSP solver: Forward Checking + MRV heuristic


In [ ]:
# We'll implement a backtracking search that uses:
# - MRV: choose unassigned variable (node) with smallest remaining domain (ties broken by degree)
# - Forward checking: after assigning, remove color from neighbors' domains; backtrack if any domain becomes empty

def make_domains(nodes, colors):
    return {n: set(colors) for n in nodes}

def select_unassigned_var(assignment, domains, graph):
    # MRV: variable with fewest available colors
    unassigned = [n for n in graph.nodes if n not in assignment]
    # If MRV tie, break by degree (highest degree first)
    best = None
    best_dom_size = None
    best_degree = None
    for v in unassigned:
        ds = len(domains[v])
        dgr = graph.degree[v]
        if best is None or ds < best_dom_size or (ds == best_dom_size and dgr > best_degree):
            best = v
            best_dom_size = ds
            best_degree = dgr
    return best

def consistent(node, color, assignment, graph):
    # Check no neighbor already has same color
    for nbr in graph.neighbors(node):
        if nbr in assignment and assignment[nbr] == color:
            return False
    return True

def forward_check(domains, node, color, graph):
    # Make a copy of domains and remove 'color' from neighbors' domains
    new_domains = {v: set(domains[v]) for v in domains}
    for nbr in graph.neighbors(node):
        if color in new_domains[nbr]:
            new_domains[nbr].remove(color)
            if len(new_domains[nbr]) == 0 and nbr not in assignment_global:
                # domain wiped out -> failure
                return None
    return new_domains

def backtrack(assignment, domains, graph, colors, max_solutions=1):
    # assignment: dict node->color
    # domains: dict node->set(colors)
    # returns a solution dict or None
    # We'll keep global count/limit so we can stop early if desired.
    if len(assignment) == graph.number_of_nodes():
        return assignment.copy()
    var = select_unassigned_var(assignment, domains, graph)
    if var is None:
        return None
    # Order values: try least-constraining-value? simple: try in order
    for color in list(domains[var]):
        if not consistent(var, color, assignment, graph):
            continue
        # Tentatively assign
        assignment[var] = color
        # Forward checking: prune neighbor domains
        saved_domains = {v: set(domains[v]) for v in domains}
        failure = False
        for nbr in graph.neighbors(var):
            if nbr not in assignment:
                if color in domains[nbr]:
                    domains[nbr].remove(color)
                    if len(domains[nbr]) == 0:
                        failure = True
                        break
        if not failure:
            result = backtrack(assignment, domains, graph, colors, max_solutions)
            if result is not None:
                return result
        # undo assignment and restore domains
        assignment.pop(var, None)
        domains = {v: set(saved_domains[v]) for v in saved_domains}
    return None

# We'll wrap with a function that sets up domains and global assignment reference used by forward_check (if used)
def solve_map_coloring(graph, colors):
    global assignment_global
    assignment_global = {}
    domains = make_domains(graph.nodes, colors)
    assignment = {}
    solution = backtrack(assignment, domains, graph, colors)
    return solution

# Define a color palette (names). Use 4 colors by default (map coloring 4-color theorem).
colors = ["red", "green", "blue", "yellow"]


## Solve and report result


In [ ]:
solution = solve_map_coloring(G, colors)
if solution is None:
    print("No solution found with colors:", colors)
else:
    print("Solution found. Assigned colors to", len(solution), "counties.")
    # Show color distribution
    dist = Counter(solution.values())
    print("Color distribution:", dist)
    # Attach color to graph nodes
    nx.set_node_attributes(G, {n: solution.get(n, None) for n in G.nodes()}, "color")
    # Print a few assignments (id, display name, color)
    sample = list(G.nodes)[:20]
    print("\nSample assignments:")
    for n in sample:
        print(n, G.nodes[n].get("display_name"), "->", G.nodes[n].get("color"))


## Visualization option A: Network graph colored nodes


In [ ]:
# This draws nodes (counties) as points, edges as adjacency. Layout computed by spring_layout.
def plot_graph_colored(G, title="County adjacency colored graph", node_size=30, show_labels=False):
    plt.figure(figsize=(12, 9))
    # layout (large graphs may take time)
    pos = nx.spring_layout(G, seed=42, k=None, iterations=200)
    node_colors = [G.nodes[n].get("color", "lightgray") for n in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_colors)
    nx.draw_networkx_edges(G, pos, alpha=0.4, width=0.4)
    if show_labels:
        labels = {n: G.nodes[n].get("display_name", str(n)) for n in G.nodes()}
        nx.draw_networkx_labels(G, pos, labels=labels, font_size=6)
    plt.title(title)
    plt.axis("off")
    plt.show()

# call it
plot_graph_colored(G, title="Map Coloring - Network View (colored nodes)", node_size=25, show_labels=False)


In [ ]:
# Cell 7 - Visualization option B: Geo plot if you have a county shapefile/GeoDataFrame
# Requirements: geopandas installed and a shapefile that contains a column matching the FIPS id field used above.
# Replace 'us_counties.shp' and 'FIPS' with your actual shapefile path and FIPS field name.

if GEO_AVAILABLE:
    # Example shapefile path and key column - change these as needed
    shapefile_path = "us_counties.shp"   # <- replace with your path
    shapefile_fips_col = "FIPS"         # <- replace with the actual column name containing fips strings/numbers

    try:
        gdf = gpd.read_file(shapefile_path)
        print("Shapefile loaded:", shapefile_path, "rows:", len(gdf))
        # Ensure fips types align with graph node ids; convert to str/int as needed
        # For this example we convert both to strings (without leading zeros) — adjust to your data
        gdf[shapefile_fips_col] = gdf[shapefile_fips_col].astype(str)
        # create dataframe from solution mapping
        sol_df = pd.DataFrame.from_dict(solution, orient='index', columns=['color'])
        sol_df.index.name = 'fipscounty'
        sol_df = sol_df.reset_index()
        sol_df['fipscounty'] = sol_df['fipscounty'].astype(str)
        # Merge with gdf on matching key - try matching by equals; you may need left-padding zeros if required
        merged = gdf.merge(sol_df, left_on=shapefile_fips_col, right_on='fipscounty', how='left')
        # Plot: counties colored by assigned color; NaN colored gray
        fig, ax = plt.subplots(1, 1, figsize=(14, 10))
        merged.plot(column='color', categorical=True, legend=True, ax=ax, missing_kwds={'color': 'lightgray'})
        ax.set_title("County coloring (geospatial) - using CSP assignment")
        ax.axis('off')
        plt.show()

    except Exception as e:
        print("Error loading or plotting shapefile:", e)
        print("Make sure geopandas is installed and shapefile path/key are correct.")
else:
    print("Geo plotting not available: geopandas not installed. Install geopandas to enable shapefile plotting.")


## Notes and troubleshooting


In [ ]:
print("""
Notes:
- If the graph is very large (thousands of counties), the simple backtracking search may be slow.
  Strategies if slow:
   * Try increasing number of colors (gives solver more flexibility).
   * Use heuristics improvements (degree heuristic tiebreaks included).
   * Solve per connected component separately.
   * Use DSATUR or heuristic graph-coloring libraries for speed (e.g., greedy_color in networkx).

- If your FIPS codes have leading zeros, ensure both CSV and shapefile FIPS fields use the same string format.
- To color by state instead of county, aggregate nodes by state and color those groups.

Feel free to paste any errors here (tracebacks) and I'll help fix them.
""")
